**JSON2RTF**

# Úvod
Cílem aplikace je vytvořit z přijatých vstupních souborů (data v JSON a formulář v rtf) vyplněný výstupní soubor rtf. Příkladem praktického využití je situace, kdy máme v JSON uloženou databázi zaměstanců a chceme z ní vytisknout určitá data - např. seznam zaměstanců, kde u každého bude uvedeno jeho bydliště. Vytvoříme si tedy pomocí MS Office Word (nebo podobného editoru) formulář, který opatříme flagy, podle kterých bude formulář vyplněn.
<br>Aplikace se skládá z knihovny JSON2RTF_lib a serveru JSON2RTF_server který implementuje JSON2RTF_lib. Server zajišťuje příjem vstupních souborů a odeslání výstupního souboru. Zároveň implementuje funkce pro zabezpečení pomocí tokenu. Knihovna pak slouží pro samotné zpracování přijatých souborů do výsledného vyplněného formuláře.
<br>Pro demonstaraci funkčnosti obsahuje aplikace testovací stránku na platformě Swagger, které umožňuje nahrát vybrané soubory a zobrazit výstup.
<br>Pro správnou funkčnost aplikce je nutné, aby uživatel byl poučený a veděl, jak správně napsat vstupní formulář - aplikace neobsahuje funkce pro kontrolu kompatibility rtf a JSON.

# Nasazení aplikace
Aplikace je z důvodu zajištění kompatibility provozována v prostředí Docker. Obsahuje jak soubor DOCKERFILE pro vytvoření image, tak i docker-compose.yaml který slouží ke spuštení kontejneru.
<br>Pro spuštění aplikace je nutné v adresáři s aplikací použít příkaz docker-compose up
<br>UI aplikace je pak defaultně na localhost:80\docs
<br>Aplikace je publikována pod MIT licencí na __[GitHub](https://github.com/MrM2266/JSON2RTF_Lib)__ a __[hub.docker.com](https://hub.docker.com/repository/docker/marek2266/json2rtf_server)__

## Docker
Pro zajištění kompatibilty je nutné mít naistalovaný Docker a Docker Compose (na Windows oba součástí Docker Hub). Adresu a port, na kterém bude aplikace dostupná v rámci kontejneru, je možné nastavit v DOCKERFILE - obsahuje parametr pro uvicorn a FastAPI.
<br>Adresu a port, na kterém bude aplikace dostupná z hostitelského pc (nebo v rámci sítě) je možné nastavit v docker-compose.yaml
<br>Pro stažení z __[hub.docker.com](https://hub.docker.com/repository/docker/marek2266/json2rtf_server)__ je možné použít příkaz: docker pull marek2266/json2rtf_server:latest

## UI
Součástí aplikace je jednoduché UI, které je ve výchozím nastavení dostupné na localhost:80/docs. Umožňuje nahrát vstupní soubory a zobrazit výstup. Jedná se o testovací prostředí vytvořené pomocí FastAPI a Swagger. Je zde také možné otestovat zabezpečení pomocí tokenu.<br>
<br> Defaultní údaje:<br>Username: jirka<br>Login: secret

# JSON file
Podle IESG Standarts Track 2070-1721 RFC8259 může JSON soubor obsahovat pouze value. Value může být: number, string, boolean, array, object a null. Každý json soubor může obsahovat jen jednu value. Jednotlivé value je možné do sebe libovolně vnořovat.
<br><br>**Pro správné zobrazení diakritiky je nutné json uložit s formátováním Windows-1250 (cp1250)**
<br>
<br>**array**
<br>- je to seřazený soubor hodnot, značí se [ ]
<br>- k hodnotám je možné přistoupit pomocí jejich indexu (key = index)
<br>- v poli mohou být pouze hodnoty oddělené čárkou - tzn. number, string, boolean, array, object a null - [10, "text",true, [ ], { }, null] - prvky pole mají jen index
<br>
<br>**object**
<br>- je to neuspořádaný soubor hodnot, značí se { }
<br>- k hodnotám je možné přistoupit pomocí jejich key
<br>- prvky jsou ve tvaru "key":"value" kde value je number, string, boolean, array, object nebo null
<br>- prvky se oddělují čárkou
<br>{
<br>	"number":8,
<br>	"string":"text",
<br>	"boolean":true,
<br>	"object":{},
<br>	"array":[],
<br>	"null":null
<br>}
<br>
<br>**number**
<br>- je to jakékoliv číslo
<br>- př. 26
<br>
<br>**string**
<br>- je to jakýkoliv text
<br>- př. "text"
<br>
<br>**boolean**
<br>- true nebo false
<br>
<br>**null**
<br>- pouze null

# Flagy v RTF
Pro označení míst, která mají být ve výtupním rtf nahrazena daty z JSON je nutné použít flagy. Flagy se píší do vstupního formuláře. Cílem flagu je označit místo, které má být nahrazeno daty z JSON a popsat, kde se požadovaná data v JSON nacházejí. JSON soubor má strukturu úrovní - je možné vnořovat libovolně prvky do sebe. Flagy definují úroveň (level), ve které se hledaná data nacházejí.
<br>
<br>
<br>Sytax flagů se řídí následujícími pravidly, která částečně vycházejí z pravidel pro JSON:
1. Nejvyšší úroveň json i rtf se nazývá root
1. Flagy se vždy píší do hranatých závorek ve tvaru [[flag:key]]
1. v rootu musí být pouze jedno pole nebo jeden objekt
1. součástí flagu je key, který obsahuje název konkrétní struktury v JSON např. [[A:pole]] - zde key = pole
1. prvky pole mají index - jako key se používá slovo null - př. [[O:null]][[E:null]] označuje objekt v poli
1. key musí být unikátní v rámci levelu
1. jako key v json nelze použít null
1. každá struktura kromě item musí mít end flag

<br><br><br>Rozlišujeme následující flagy:
<br>
<br>**end**
<br>[[E:key]]
<br>- označuje konec struktury
<br>
<br>**object**
<br>[[O:key]] - označuje začátek objektu key - program vyhledá v aktuální úrovni json souboru objekt auto a vstoupí do něj
<br>[[E:key]] - označuje konec objektu key - program ukončí hledání v objektu auto a přesune se v json souboru o úroveň nahoru
<br>- odpovídá hodnotě object v json
<br>
<br>**array**
<br>[[A:key]] - označuje začátek pole key - program přejde z aktuálního levelu v json souboru do pole key
<br>[[E:key]] - označuje konec pole key - program se přesune v json souboru o level nahoru
<br>- Flag array je specifický tím, že kód, který je mezi start flagem [[A:lide]] a end flagem [[E:lide]] se provede pro každý prvek pole.
<br>- Program spočítá počet prvků v json (např. pole lide obsahuje v json 5 objektů) a provede zadaný kód 5x -> kód mezi [[A:key]] a [[E:key]] se provede 5x po sobě, pokaždé pro jeden prvek pole key
<br>- odpovídá hodnotě array v json
<br>
<br>**item**
<br>[[I:key]]
<br>- označuje místo, které se má nahradit z json konkrétními daty - program v json objektu najde položku s názvem key a místo [[I:key]] doplní data z json
<br>- odpovídá hodnotám number, string, boolean, null v json
<br>
<br>
<br>
<br>Možné kombinace a jejich zápis pomocí flagů:
<br>
<br>**Root může obsahovat:**
<br>[[A:root]] - nepojmenované pole v rootu; end flag [[E:root]]
<br>[[O:root]] - nepojmenovaný objekt v rootu;end flag [[E:root]]
<br>
<br>**Array může obsahovat:**
<br>[A:null]] - nepojmenované pole v poli; end flag [[E:null]]
<br>[[O:null]] - nepojmenovaný objekt v poli; end flag [[E:null]]
<br>[[I:index]] - index čísla, stringu nebo booleanu pro vypsání
<br>
<br>**Object může obsahovat:**
<br>[[A:key]] - pro pole v objektu - musí být pojmenované (musí mít strukturu key:value); end flag [[E:key]]
<br>[[O:key]] - pro objekt v objektu - musí bý pojmenovaný; end flag [[E:key]]
<br>[[I:key]] - pro string, číslo, boolean v objektu - musí být pojmenovaný
<br>
<br>**Vypsání všech prvků pole:**
<br>- v json máme pole, které obsahuje pouze stringy, nebo čísla - chceme celé pole vypsat
<br>- pomocí flagů [[A:key]][[E:key]] spustíme na poli key fci ArrayAsString, která vypíše prvky pole jako stringy oddělené čárkami

# Postup při vytváření šablony pro vyplnění
1. V MS Word vytvoříme šablonu - s formátováním, tabulkami atd. - uložíme jako rtf
1. Šablonu otevřeme v notepadu - dopíšeme flagy a uložíme jako rtf
1. odešleme spolu s příslušným JSON na server<br><br><br>

# Popis funkcionality knihovny JSON2RTF_lib
Tato kapitola se bude věnovat hlubšímu popisu knihovny. Bude ilustrovat jednotlivé funkce a popíše způsob, jakým jsou soubory zpracovány.

## Obecně
Knihovna obsahuje dvě základní třídy CData a CjsonReader. Třída CData má dvě instance - input a output. Slouží pro uložení vstupního a výstupního rtf jako string. Třída má proměnnou m_data, která obsahuje rtf. Poskytuje základní metody - LoadData a Add. Díky nim umí input i output načíst data a přidat si do m_data část vygenerovaného rtf.
<br>Třída CjsonReader zajišťuje "pohyb" v json souboru a čtení dat z něj. Poskytuje metody Down, Up a NextElement  pro "pohyb" v levelech JSON a metody GetArraySize a GetRootSize pro zjištění velikosti polí. Metody GetArrayAsString a GetRootAsString vracejí prvky pole jako string. Metoda GetItem pak vrací data z JSON objektu.
<br><br>Obecně knihovna funguje tak, že celý rtf soubor se předá do fce Process. Ta najde první start flag v řetezci input. Vše co je před start flagem přidá do output. K start flagu najde odpovídající end flag. Vše co je před end flagem odstarní z input (data jsou zpracována). Podle flagu (A, O, I) spustí odpovídající funkci - Array, Object, Item a předá jí vše, co je mezi start flagem a end flagem. Funkce spuštná podle flagu (Array, Object, Item) opět předává svou část do fce Process -> je možné jít do libovolné úrovně JSON - JSON se neustále "rozbaluje".
<br>Funkce se takto volají neustále mezi sebou, až string m_data v input neobsahuje žádné znaky - výsledný rtf je zpracovaný.

## Hledání Flagů
Chceme zpracovat rtf soubor uložený ve stringu inputRTF

In [45]:
inputRTF = "rtf kod se speciálními znaky ,.ů§/*86 [[O:auto]] kod objektu auto[[E:auto]] pokračování rtf kódu"

<br>Nejprve si definujeme třídu CData, jejíž instance budou input a output - bude se starat o vstupní a výstupní rtf.

In [46]:
class CData:
    def __init__(self):
        self.m_data=""

    def Add(self, add):
        """Gets a string that is added to m_data
	
	    Args:
		add: value that is added to m_data
	    """
        self.m_data += str(add)

    def LoadData(self, data):
        """Takes rtf file as a string and stores it into m_data
	
	    Args:
		    data: rtf as string - from fastAPI
	    """
        self.m_data = data

<br>Vyhledáme první start flag - fce FlagStart vrací array ve tvaru [flag, key, počátek start flagu, konec start flagu]. String inpurRTF získáme např. z FastAPI

In [47]:
import re

def FlagStart(data):
     """Finds the first flag in a string of data and finds out information  
     
     Information: What flag is it, it's key, beginning and end
	  
     Args:
         data: String of data you want to find the information about
     
     Returns:
         [flag, key, match.start(), match.end()]: if the functions finds out the information
         None: if the function doesn't find out any information  
     """
     match = re.search("\[{2}((A:[a-zA-Z0-9]+)|(O:[a-zA-Z0-9]+)|(I:[a-zA-Z0-9]+))\]{2}", data)
     if match:
         flag = data[match.start() + 2 : match.start() + 3]
         key = data[match.start() + 4 : match.end() - 2]
         return [flag, key, match.start(), match.end()]
     else:
         return None

input = CData()
output = CData()
input.LoadData(inputRTF)

outputStart = FlagStart(input.m_data)

print(f"Flag: {outputStart[0]}")
print(f"Key: {outputStart[1]}")
print(f"Pozice prvního znaku start flagu: {outputStart[2]}")
print(f"Pozice posledního znaku start flagu: {outputStart[3]}")

Flag: O
Key: auto
Pozice prvního znaku start flagu: 38
Pozice posledního znaku start flagu: 48


<br>Funkce vyhledala v řetězci první start flag a jako výstup předává všechny potřebné informace - key, pozice prvního a posledního znaku.
<br>Pomocí funkce FlagEnd vyhledáme na zadaném řetězci end flag s konkrétním key

In [48]:
def FlagEnd(data, key):
    """Returns the positon of the ending tag
	
    Args:
        data: String of data in which you want to find the position of the ending tag
        key: Value of the ending tag

    Returns:
        Positon of the ending tag
        Beginning and ending of the tag
    """
    if (key == "null"):
        list = []
        start=[]
        end=[]

        for match in re.finditer("\[{2}(A|O):null\]{2}", data):
            s = match.start()
            list.append(s)
            start.append(s)
        for match in re.finditer("\[{2}E:null\]{2}", data):
            s = match.start()
            list.append(s)
            end.append(s)

        list.sort()
        count=0

        for i in list:
            if i in start:
                count += 1
            if i in end:
                count -= 1
            if (count == 0):
                return [i,i+10]
    else:
        str = "[[E:" + key + "]]"
        return [data.find(str), data.find(str) + len(str)]
    

outputEnd = FlagEnd(inputRTF, outputStart[1])

print(f"Pozice prvního znaku end flagu: {outputEnd[0]}")
print(f"Pozice posledního znaku end flagu: {outputEnd[1]}")

Pozice prvního znaku end flagu: 65
Pozice posledního znaku end flagu: 75


<br><br>Výstup z obou funkcí používá funkce Process, která předá část vstupního rtf do výstupu, odstraní flagy a kód mezi nimi předá do příslušné funkce. Funkce jsou uvedené v modifikované podobě - pouze zobrazují data, která přijímají jako parametry.

In [49]:
def Process(data):
    """Takes string data and finds the first tag from the beginning (start tag) and corresponding end tag
    Text that is before start flag is added to output
    Text that is after end flag is left for another loop
    The code between the start flag and the end flag is passed into Process function again

    Args:
        data: string to process

    Returns:
        str: text that is after the end flag - code for another loop
    """

    start = FlagStart(data)
    if start != None:
        output.Add(data[0:start[2]])
    
        if start[0] != "I":
            end = FlagEnd(data, start[1])
            if start[0] == "A":
                Array(data[start[3]:end[0]], start[1])
            if start[0] == "O":
                Object(data[start[3]:end[0]], start[1])
            return data[end[1]:]
        else:
            Item(start[1])
            return data[start[3]:]
    else:
        output.Add(data)
        return ""
    
def Array(data, key):
    print("Funkce Array\n=============================================")
    print(f"Key k vyhledání v json: {key}")
    print(f"Kód pole: {data}")

    

def Object(data, key):
    print("Funkce Object\n=============================================")
    print(f"Key k vyhledání v json: {key}")
    print(f"Kód objektu: {data}")


def Item(key):
    print("Funkce Key\n=============================================")
    print(f"Key k vyhledání v json: {key}")


input.m_data = Process(input.m_data)

Funkce Object
Key k vyhledání v json: auto
Kód objektu:  kod objektu auto


<br>V tuto chvíli jsme schopni dekódovat první úroveň flagů. Funkce Array a Object ve skutečnosti nezobrazují výstup, ale znovu svůj kód předají do funkce Process - v kódu např. objektu dojde k vyhledání dalších struktur a spuštění příslušných funkcí - v objektu je např. pole -> funkce Process spustí funkci Array - ta opět provede analýzu svého kódu pomocí Process -> kód může jít do libovolné úrovně.
<br>V tomto příkladu popíšeme pouze práci s první úrovní - inputRTF obsahuje pouze jednu dvojici start flag - end flag<br><br>

## Vyhledávání v JSON
Budeme ilustrovat funkci třídy CjsonReader na stringu inputJson, který obsahuje vstupní JSON soubor (např. z FastAPI)

In [50]:
inputJson = """
{
  "states": [
    {
      "name": "Alabama",
      "abbreviation": "AL",
      "areaCodes": [ "205", "251", "256", "334", "938" ]
    },
    {
      "name": "Alaska",
      "abbreviation": "AK",
      "areaCodes": [ "907" ]
    },
    {
      "name": "Arizona",
      "abbreviation": "AZ",
      "areaCodes": [ "480", "520", "602", "623", "928" ]
    },
    {
      "name": "Arkansas",
      "abbreviation": "AR",
      "areaCodes": [ "479", "501", "870" ]
    }]
}
"""

Definujeme si třídu CjsonReader, který bude zajišťovat práci s JSON souborem a vyhledávání v něm.

In [51]:
import json

class CjsonReader:
    def __init__(self):
        self.m_levels = [] #list of individual levels, current is always the last item of the array
        self.m_level = 0
        self.m_indexes = [0] #list of indexes (index 0 contains value 2 -> I'am on an index 2 on level 0

    def LoadData(self, jsonString, decode):
        """Takes JSON as bytes. If decode is 1 JSON is decoded using ANSI and then stored into m_levels.

        Args:
            jsonString: jsonFile as bytes - from fastAPI
            decode: parameter; if 1 -> string is decoded using cp1250; if 0 -> string is stored without decoding
        """
        if (decode == 1):
            self.m_levels.append(json.loads(str(jsonString, 'cp1250')))
        else:
            self.m_levels.append(json.loads(jsonString))

    def Down(self, key):
        """Gets you down a level to entered key or index
        
	    Args:
		    key: Index or key you want to step down to 
	    """
        self.m_levels.append(self.m_levels[self.m_level][key])
        self.m_indexes.append(0)
        self.m_level += 1

    def Up(self):
        """Gets you up a level
	    """
        if self.m_level > 0:
            del self.m_levels[-1]
            del self.m_indexes[-1]
            self.m_level -= 1

    def GetItem(self, key):
        """Returns string from JSON dictionary
	
	    Args:
	        key: JSON dictionary key to find desired string

	    Returns:
		    str: string found under parameter key
        """
        return self.m_levels[self.m_level][key]

    def GetArraySize(self, key):
        """Returns the size of an array
	
	    Args:
            key: Key of an array you want to get the size of

	    Returns:
		    int: Size of an array
        """
        return len(self.m_levels[self.m_level][key])

    def GetRootSize(self):
        """Returns the size of root array

	    Returns:
		    int: Size of root
        """
        return len(self.m_levels[0])

    def GetArrayAsStr(self, key):
        """Returns array as one string - elements are separated with ,
	
	    Args:
            key: Key of an array you want to get as a string

	    Returns:
		    int: Size of anarray
        """
        result = ""
        for i in self.m_levels[self.m_level][key]:
            result = result + str(i) + ", "
        return result[:-2]

    def GetRootAsStr(self):
        """Returns root array as one string - elements are separated with ,
	
	    Args:
            key: Key of an array you want to get as a string

	    Returns:
		    int: Size of root
        """
        result = ""
        for i in self.m_levels[0]:
            result = result + str(i) + ", "
        return result[:-2]

    def NextElement(self):
        """Moves you one item forward in current level (only in array)
        """
        self.m_indexes[self.m_level] += 1

    def GetIndex(self):
        """Returns index of processing element on current level

        Returns:
            int: index of an element
        """
        return self.m_indexes[self.m_level]

V následujícím kódu si vytvoříme instanci třídy CjsonReader a pomocí jejích metod budeme zobrazovat data z inputJson.

In [52]:
jsonData = CjsonReader()
jsonData.LoadData(inputJson, 0)

jsonData.Down("states") ##v souboru je objekt, který obsahuje pole states; z root objektu vstoupíme do states
jsonData.Down(0) ##v poli states vstoupíme do prvku 0

print("Data z prvku 0\n==================================")
print(jsonData.GetItem("name")) ##vypíšeme položku name státu s indexem 0
print(jsonData.GetItem("abbreviation"))
print(jsonData.GetArrayAsStr("areaCodes")) ##vrací pole jako string

jsonData.Up() ##vrátíme se o úroveň nahoru - z prvku 0 do pole states
jsonData.Down(1) ##vstoupíme do prvku 1

print("\n\nData z prvku 1\n==================================")
print(jsonData.GetItem("name")) ##vypíšeme položku name státu s indexem 0
print(jsonData.GetItem("abbreviation"))
print(jsonData.GetArrayAsStr("areaCodes")) ##vrací pole jako string

jsonData.Up() ##vrátíme se o úroveň nahoru - z prvku 1 do pole states
jsonData.Down(2) ##vstoupíme do prvku 2

print("\n\nData z prvku 2\n==================================")
print(jsonData.GetItem("name")) ##vypíšeme položku name státu s indexem 0
print(jsonData.GetItem("abbreviation"))
print(jsonData.GetArrayAsStr("areaCodes")) ##vrací pole jako string

jsonData.Up() ##jsme v poli states


print("\n\nSeznam států v json\n==================================")
jsonData.Up()
pocet = jsonData.GetArraySize("states") ##vrací velikost pole states
jsonData.Down("states")

for i in range(0,pocet):
    jsonData.Down(i)
    print(jsonData.GetItem("name"))
    jsonData.Up()

Data z prvku 0
Alabama
AL
205, 251, 256, 334, 938


Data z prvku 1
Alaska
AK
907


Data z prvku 2
Arizona
AZ
480, 520, 602, 623, 928


Seznam států v json
Alabama
Alaska
Arizona
Arkansas


<br><br>
## Generování výstupu
Knihovna JSON2RTF_lib propojuje obě třídy dohromady a podle toho, jak čte rtf flagy, tak provádí čtení z JSON souboru. V následujícím kódu bude ukázána funkce celé knihovny. Na vstupu budeme mít dva stringy - jeden obsahuje data z vdtupního rtf formuláře a druhý obsahuje data z JSON.
<br>V kódu bude použita hotová knihovna JSON2RTF_lib.py tak, jak jí používá JSON2RTF_server.

In [53]:
%reset -f

inputJson = """
{
  "states": [
    {
      "name": "Alabama",
      "abbreviation": "AL",
      "areaCodes": [ "205", "251", "256", "334", "938" ]
    },
    {
      "name": "Alaska",
      "abbreviation": "AK",
      "areaCodes": [ "907" ]
    },
    {
      "name": "Arizona",
      "abbreviation": "AZ",
      "areaCodes": [ "480", "520", "602", "623", "928" ]
    },
    {
      "name": "Arkansas",
      "abbreviation": "AR",
      "areaCodes": [ "479", "501", "870" ]
    }]
}
"""

In [54]:
inputRTF = "[[O:root]]Seznam statu \n================\n[[A:states]][[O:null]]Stat: [[I:name]]\n[[E:null]][[E:states]][[E:root]]"
#inputRTF = "[[O:root]]Seznam statu \n================\n\n[[A:states]][[O:null]]Stat: [[I:name]]\nKod: [[I:abbreviation]]\nOblasti: [[A:areaCodes]][[E:areaCodes]]\n\n[[E:null]][[E:states]][[E:root]]"

In [55]:
import JSON2RTF_lib as RTF

RTF.Init()
RTF.LoadJson(inputJson, 0)
RTF.LoadRTF(inputRTF)

RTF.ProcessRTF()

outputRTF = RTF.GetOutput()

print(outputRTF)

Seznam statu 
Stat: Alabama
Stat: Alaska
Stat: Arizona
Stat: Arkansas

